# Análisis de Impacto Meteorológico en el Tráfico Móvil

Este notebook analiza si existe una correlación entre las precipitaciones (lluvia) y el volumen de tráfico de telecomunicaciones en Milán.

## Datos Utilizados
- **Tráfico**: Datasets originales (`data1.csv`, `data2.csv`).
- **Clima**: Datos históricos de Open-Meteo para Milán (Nov-Dic 2013).

In [ ]:
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuración de Gráficos
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 6)

## 1. Cargar Datos Climáticos

In [ ]:
# Cargar datos de clima generados previamente
df_weather = pd.read_csv('clima_milano.csv')
df_weather['time'] = pd.to_datetime(df_weather['time'])
df_weather.set_index('time', inplace=True)

print("Datos de clima cargados:")
print(df_weather.head())

## 2. Cargar y Procesar Datos de Tráfico

In [ ]:
files = ['../data1.csv/data1.csv', '../data2.csv/data2.csv']

# Leer columnas necesarias
df = dd.read_csv(files, assume_missing=True, 
                 usecols=['TimeInterval', 'smsin', 'smsout', 'callin', 'callout', 'internet'])

# Calcular tráfico total
df['TotalTraffic'] = df['smsin'] + df['smsout'] + df['callin'] + df['callout'] + df['internet']

# Agrupar por hora (TimeInterval está en ms)
traffic_series = df.groupby('TimeInterval')['TotalTraffic'].sum().compute()
traffic_series.index = pd.to_datetime(traffic_series.index, unit='ms')

# Resamplear a horario para coincidir con clima (si no lo está ya)
traffic_hourly = traffic_series.resample('h').sum()

print("Tráfico procesado (primeras 5 filas):")
print(traffic_hourly.head())

## 3. Unir Datasets

In [ ]:
# Unir por índice (Tiempo)
merged_df = pd.merge(traffic_hourly, df_weather, left_index=True, right_index=True, how='inner')

# Filtrar lluvia relevante (> 0.1 mm)
merged_df['IsRainy'] = merged_df['rain'] > 0.1

print(f"Registros totales: {len(merged_df)}")
print(f"Horas con lluvia: {merged_df['IsRainy'].sum()}")

## 4. Análisis Visual: Tráfico con Lluvia vs Sin Lluvia

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='IsRainy', y='TotalTraffic', data=merged_df)
plt.title('Distribución del Tráfico: Lluvia vs Seco')
plt.xlabel('¿Está lloviendo? (False=Seco, True=Lluvia)')
plt.ylabel('Volumen de Tráfico Total')
plt.show()

## 5. Correlación

In [ ]:
corr = merged_df['TotalTraffic'].corr(merged_df['rain'])
print(f"Coeficiente de Correlación (Tráfico vs Lluvia): {corr:.4f}")

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(merged_df['rain'], merged_df['TotalTraffic'], alpha=0.5)
plt.title(f'Scatter Plot: Lluvia vs Tráfico (Corr: {corr:.2f})')
plt.xlabel('Lluvia (mm)')
plt.ylabel('Tráfico Total')
plt.grid(True)
plt.show()